In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences
from keras.models import load_model

In [2]:
df = pd.read_csv('..\\dataset\\spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [8]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [9]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [10]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [12]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 5s 169ms/step - loss: 0.0407 - accuracy: 0.9886 - val_loss: 0.0743 - val_accuracy: 0.9768
Epoch 2/10
30/30 [==============================] - 5s 160ms/step - loss: 0.0286 - accuracy: 0.9910 - val_loss: 0.0597 - val_accuracy: 0.9821
Epoch 3/10
30/30 [==============================] - 5s 153ms/step - loss: 0.0231 - accuracy: 0.9934 - val_loss: 0.0714 - val_accuracy: 0.9810
Epoch 4/10
30/30 [==============================] - 4s 150ms/step - loss: 0.0170 - accuracy: 0.9955 - val_loss: 0.0631 - val_accuracy: 0.9842
Epoch 5/10
30/30 [==============================] - 5s 150ms/step - loss: 0.0123 - accuracy: 0.9966 - val_loss: 0.0730 - val_accuracy: 0.9842
Epoch 6/10
30/30 [==============================] - 4s 148ms/step - loss: 0.0096 - accuracy: 0.9976 - val_loss: 0.0818 - val_accuracy: 0.9863
Epoch 7/10
30/30 [==============================] - 5s 154ms/step - loss: 0.0069 - accuracy: 0.9982 - val_loss: 0.0898 - val_accuracy: 0.9831
Epoch 

In [15]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 22ms/step - loss: 0.1366 - accuracy: 0.9844


[0.13660170137882233, 0.9844497442245483]

In [16]:
model.save("..\\models\\sms.keras")

In [20]:
model1 = load_model("..\\models\\sms.keras")

In [21]:
model1.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 21ms/step - loss: 0.1366 - accuracy: 0.9844


[0.13660170137882233, 0.9844497442245483]